In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import pandas as pd
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.lfr import LFR
# Import scikit-learn core slibraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Taiwan-Credit-Card-Cleaned-Marriage dataset
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned-Gender.csv')

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'GENDER'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'DEFAULT'
fav_label           = 1 # Will not default next month
unfav_label         = 0 # Will default next month
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create a Binary Label Dataset to use with AIF360 APIs
X = Taiwan_df.drop(lab,axis=1)
y = Taiwan_df[lab]

In [5]:
Taiwan_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
Taiwan_train_df, Taiwan_test_df = Taiwan_bld.split([0.8], shuffle=True, seed=101)

In [7]:
# Scale the train and test datasets
scaler = MinMaxScaler(copy=False)
Taiwan_train_df.features = scaler.fit_transform(Taiwan_train_df.features)
Taiwan_test_df.features  = scaler.fit_transform(Taiwan_test_df.features)

In [8]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
Taiwan_train, d = Taiwan_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test, d = Taiwan_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = Taiwan_train.drop(lab,axis=1)
y_train = Taiwan_train[lab]
X_test = Taiwan_test.drop(lab,axis=1)
y_test = Taiwan_test[lab]
biasedLogModel = LogisticRegression(random_state=101)
biasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
biasedLogModel.fit(X_train, y_train) 
biasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {biasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {biasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.8083333333333333
Random Forest       validation accuracy: 0.7946666666666666


In [9]:
# Create the binary label dataset metric class for the train dataset
metric_train_bld = BinaryLabelDatasetMetric(Taiwan_train_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())
print('Number of instances           :', metric_train_bld.num_instances())
print("Base Rate                     :%f" % metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_bld.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_bld.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_bld.num_positives(privileged=True)+metric_train_bld.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_bld.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_bld.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_bld.num_negatives(privileged=True)+metric_train_bld.num_negatives(privileged=False))
display(Markdown("#### Biased training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

#### Biased training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.033271
Number of instances           : 24000.0
Base Rate                     :0.780625
Consistency                   : [0.78445833]
Disparate Impact              : 1.0437451995559814
Mean Difference               : 0.03327060860999975
Statistical Parity Difference : 0.03327060860999975
# of positives(privileged)    : 7242.0
# of positives(non-privileged): 11493.0
Total positive instances"     : 18735.0
# of negatives(privileged)    : 2280.0
# of negatives(non-privileged): 2985.0
Total negative instances"     : 5265.0


#### Biased training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.033271


In [10]:
# Fit the Learning Fair Representations on the biased training data
print('Interval : Optimization objective value for the interval')
TR = LFR(unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups,verbose=1, seed=101)
TR = TR.fit(Taiwan_train_df)

Interval : Optimization objective value for the interval
250 14578.08549718056
500 13605.03020050427
750 13083.612646790078
1000 13036.594166704075
1250 12954.67519081236
1500 12732.308566374204
1750 18089.89559272153
2000 12580.510148898644
2250 13388.197792134097
2500 12366.493271331
2750 12348.177285776956
3000 12308.01642107602
3250 12023.019250304562
3500 12905.484141530787
3750 12356.352453873673
4000 11975.145094362962
4250 11851.562431848142
4500 13695.972154762827
4750 11726.462035654917
5000 11681.679846702305


In [11]:
# Transform training data and align features
Taiwan_train_lfr = TR.transform(Taiwan_train_df)

In [12]:
# Determine the transformed model accuracy for Logistic Regression and Random Forest Classifiers
# Convert the scaled Binary Labelled Dataset to a pandas dataframe for consistency 
Taiwan_train, d = Taiwan_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = Taiwan_train.drop("DEFAULT",axis=1)
y_train = Taiwan_train["DEFAULT"]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.7935
Random Forest       validation accuracy: 0.7908333333333334


In [13]:
display(Markdown("#### Transformed training dataset"))
from sklearn.metrics import classification_report
thresholds = [0.1, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7]
for threshold in thresholds:  
    # Transform training data and align features
    Taiwan_train_lfr = TR.transform(Taiwan_train_df,threshold=threshold)
    metric_train_lfr = BinaryLabelDatasetMetric(Taiwan_train_lfr, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    print("Classification threshold = %f" % threshold)
    #print(classification_report(dataset_orig_train.labels, dataset_transf_train.labels))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())

#### Transformed training dataset

Classification threshold = 0.100000
Difference in mean outcomes between unprivileged and privileged groups = 0.000177
Classification threshold = 0.300000
Difference in mean outcomes between unprivileged and privileged groups = 0.002599
Classification threshold = 0.350000
Difference in mean outcomes between unprivileged and privileged groups = 0.004116
Classification threshold = 0.400000
Difference in mean outcomes between unprivileged and privileged groups = 0.007013
Classification threshold = 0.500000
Difference in mean outcomes between unprivileged and privileged groups = 0.016431
Classification threshold = 0.600000
Difference in mean outcomes between unprivileged and privileged groups = 0.034877
Classification threshold = 0.700000
Difference in mean outcomes between unprivileged and privileged groups = 0.072923


In [14]:
print('THIS IS WHAT THE AIF CODE PRODUCES FROM THE LAST CLASSIFICATION THRESHOLD RUN ABOVE')
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())
print('Number of instances           :', metric_train_lfr.num_instances())
print("Base Rate                     :%f" % metric_train_lfr.base_rate())
print('Consistency                   :', metric_train_lfr.consistency())
print('Disparate Impact              :', metric_train_lfr.disparate_impact())
print('Mean Difference               :', metric_train_lfr.mean_difference())
print('Statistical Parity Difference :', metric_train_lfr.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_lfr.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_lfr.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_lfr.num_positives(privileged=True)+metric_train_lfr.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_lfr.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_lfr.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_lfr.num_negatives(privileged=True)+metric_train_lfr.num_negatives(privileged=False))
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

THIS IS WHAT THE AIF CODE PRODUCES FROM THE LAST CLASSIFICATION THRESHOLD RUN ABOVE
Difference in mean outcomes between unprivileged and privileged groups = 0.072923
Number of instances           : 24000.0
Base Rate                     :0.863042
Consistency                   : [0.99704167]
Disparate Impact              : 1.0890340696871728
Mean Difference               : 0.07292340994436686
Statistical Parity Difference : 0.07292340994436686
# of positives(privileged)    : 7799.0
# of positives(non-privileged): 12914.0
Total positive instances"     : 20713.0
# of negatives(privileged)    : 1723.0
# of negatives(non-privileged): 1564.0
Total negative instances"     : 3287.0


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.033271


In [15]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_train_lfr.consistency())
print("Consistency of labels in original training dataset= %f" %metric_train_bld.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 0.997042
Consistency of labels in original training dataset= 0.784458


In [16]:
# Compare the original and transformed datasets
#Convert the returned Binary Labelled Dataset to a pandas dataframe 
Taiwan_orig_df, d = Taiwan_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_lfr_df, d = Taiwan_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
# Check whether the transform on the original dataset has worked. 
# A false means that the dataset is transformed.

Taiwan_lfr_df.equals(Taiwan_df)

False

In [17]:
Taiwan_lfr_df.head(3)

,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
6015,0.308190,0.0,0.373589,0.504044,0.256706,-0.344691,-0.094310,0.025233,-0.008984,0.033267,...,0.216855,0.125657,0.503885,-0.083071,-0.220214,-0.171899,-0.204767,-0.109420,-0.100871,1.0
28660,0.301151,0.0,0.406784,0.523528,0.294951,-0.091104,0.066368,0.229157,0.132202,0.106698,...,0.282134,0.192730,0.511756,0.052980,-0.009526,-0.102760,-0.044746,0.003198,0.015386,1.0
12498,0.284733,1.0,0.397808,0.511513,0.290347,-0.109891,0.056963,0.202343,0.128835,0.114426,...,0.246418,0.169629,0.514724,0.003549,-0.039784,-0.117320,-0.078252,-0.036648,0.003845,1.0


In [18]:
Taiwan_orig_df.head(3)

,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
6015,0.063291,0.0,0.333333,0.333333,0.482759,0.3,0.0,0.000000,0.0,0.000000,...,0.193873,0.089884,0.373913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
28660,0.063291,0.0,0.666667,0.666667,0.396552,0.2,0.2,0.222222,0.2,0.222222,...,0.229724,0.125373,0.410898,0.002633,0.002119,0.001800,0.001932,0.004689,0.002459,1.0
12498,0.050633,1.0,0.666667,0.333333,0.465517,0.2,0.2,0.222222,0.2,0.111111,...,0.197500,0.092570,0.374772,0.002061,0.001227,0.000007,0.003913,0.001829,0.000000,1.0


In [19]:
Taiwan_orig_df = Taiwan_orig_df.reset_index(drop=True)
Taiwan_lfr_df  = Taiwan_lfr_df.reset_index(drop=True)
TaiwanBool     = (Taiwan_orig_df != Taiwan_lfr_df).stack()  # Create Frame of comparison booleans
Taiwandiff     = pd.concat([Taiwan_orig_df.stack()[TaiwanBool], Taiwan_lfr_df.stack()[TaiwanBool]], axis=1)
Taiwandiff.columns=["Taiwan_df", "Taiwan_lfr"]
print(Taiwandiff)

                 Taiwan_df  Taiwan_lfr
0     LIMIT_BAL   0.063291    0.308190
      EDUCATION   0.333333    0.373589
      MARRIAGE    0.333333    0.504044
      AGE         0.482759    0.256706
      PAY_1       0.300000   -0.344691
      PAY_2       0.000000   -0.094310
      PAY_3       0.000000    0.025233
      PAY_4       0.000000   -0.008984
      PAY_5       0.000000    0.033267
      PAY_6       0.000000   -0.125388
      BILL_AMT1   0.181479    0.120185
      BILL_AMT2   0.085748    0.122078
      BILL_AMT3   0.184931    0.200536
      BILL_AMT4   0.193873    0.216855
      BILL_AMT5   0.089884    0.125657
      BILL_AMT6   0.373913    0.503885
      PAY_AMT1    0.000000   -0.083071
      PAY_AMT2    0.000000   -0.220214
      PAY_AMT3    0.000000   -0.171899
      PAY_AMT4    0.000000   -0.204767
      PAY_AMT5    0.000000   -0.109420
      PAY_AMT6    0.000000   -0.100871
      DEFAULT     0.000000    1.000000
1     LIMIT_BAL   0.063291    0.301151
      EDUCATION   0.66666

In [20]:
## PCA Analysis of consitency

In [21]:
# At this stage the transformed dataframe will have the last threshold encountered!
print('this is the threshold being used :',threshold)
feat_cols = Taiwan_train_df.feature_names

orig_df = pd.DataFrame(Taiwan_train_df.features,columns=feat_cols)
orig_df['label'] = Taiwan_train_df.labels
orig_df['label'] = orig_df['label'].apply(lambda i: str(i))

transf_df = pd.DataFrame(Taiwan_train_lfr.features,columns=feat_cols)
transf_df['label'] = Taiwan_train_lfr.labels
transf_df['label'] = transf_df['label'].apply(lambda i: str(i))

this is the threshold being used : 0.7


In [22]:
from sklearn.decomposition import PCA

orig_pca = PCA(n_components=3)
orig_pca_result = orig_pca.fit_transform(orig_df[feat_cols].values)

orig_df['pca-one'] = orig_pca_result[:,0]
orig_df['pca-two'] = orig_pca_result[:,1] 
orig_df['pca-three'] = orig_pca_result[:,2]

display(Markdown("#### Original training dataset"))
print('Explained variation per principal component:')
print(orig_pca.explained_variance_ratio_)

#### Original training dataset

Explained variation per principal component:
[0.472332   0.14967459 0.11719392]


In [23]:
transf_pca = PCA(n_components=3)
transf_pca_result = transf_pca.fit_transform(transf_df[feat_cols].values)

transf_df['pca-one'] = transf_pca_result[:,0]
transf_df['pca-two'] = transf_pca_result[:,1] 
transf_df['pca-three'] = transf_pca_result[:,2]

display(Markdown("#### Transformed training dataset"))
print('Explained variation per principal component:')
print(transf_pca.explained_variance_ratio_)

#### Transformed training dataset

Explained variation per principal component:
[0.93394652 0.05821425 0.00783895]


In [24]:
# Test whether we can predict the Sensitive (Protected) variable from the transformed training dataset
X_train = Taiwan_lfr_df.drop(protected_attr,axis=1)
y_train = Taiwan_lfr_df[protected_attr]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
# Now test whether we can predict Gender from the test dataset
Taiwan_test, d = Taiwan_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_test = Taiwan_test.drop(protected_attr,axis=1)
y_test = Taiwan_test[protected_attr]
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.43066666666666664
Random Forest       validation accuracy: 0.45966666666666667


In [25]:
###Load, clean up original test data and compute metric

In [26]:
display(Markdown("#### Testing Dataset shape"))
print(Taiwan_test_df.features.shape)

metric_test_bld = BinaryLabelDatasetMetric(Taiwan_test_df, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test_bld.mean_difference())

#### Testing Dataset shape

(6000, 23)


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.036545


In [27]:
###Transform test data and compute metric

In [28]:
Taiwan_test_lfr = TR.transform(Taiwan_test_df, threshold=threshold)
metric_test_lfr = BinaryLabelDatasetMetric(Taiwan_test_lfr, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

In [29]:
print("Consistency of labels in tranformed test dataset= %f" %metric_test_lfr.consistency())

Consistency of labels in tranformed test dataset= 0.996633


In [30]:
print("Consistency of labels in original test dataset= %f" %metric_test_bld.consistency())

Consistency of labels in original test dataset= 0.777433


In [31]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_test_lfr.consistency() > metric_test_bld.consistency(), "Transformed dataset consistency should be greater than original dataset."

print(check_algorithm_success())

None
